In [1]:
import os

from openai import OpenAI, Stream
from openai.types.beta.assistant_stream_event import AssistantStreamEvent

from timestep.config import Settings

settings = Settings()

In [2]:
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY", settings.openai_api_key),
    base_url=os.getenv("OPENAI_BASE_URL", settings.openai_base_url),
)

In [3]:
assistant = client.beta.assistants.create(
    model="gpt-4o"
)

In [4]:
tools = [
  {
    "type": "function",
    "function": {
      "name": "get_current_weather",
      "description": "Get the current weather in a given location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "The city and state, e.g. San Francisco, CA",
          },
          "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
        },
        "required": ["location"],
      },
    }
  }
]

stream: Stream[AssistantStreamEvent] = client.beta.threads.create_and_run(
  thread={
      "messages": [
        {"role": "user", "content": "What is the weather like in San Francisco?"}
      ]
  },
  # assistant_id="asst_abc123",
  assistant_id=assistant.id,
  tools=tools,
  stream=True
)

try:
  for event in stream:
    print(event)

finally:
  client.beta.assistants.delete(assistant.id)
  stream.close()

ThreadRunCreated(data=Run(id='65d8cde1-cfd3-45e0-a4db-81081b6bfeb8', assistant_id='484fc52c-4d45-4691-8cb2-68577dca0e7e', cancelled_at=None, completed_at=None, created_at=1723655592, expires_at=None, failed_at=None, incomplete_details=None, instructions='', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata=None, model='gpt-4o', object='thread.run', parallel_tool_calls=False, required_action=None, response_format=None, started_at=None, status='queued', thread_id='834922cd-87c5-447b-9393-6b2da2c56fdf', tool_choice=None, tools=[FunctionTool(function=FunctionDefinition(name='get_current_weather', description='Get the current weather in a given location', parameters={'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city and state, e.g. San Francisco, CA'}, 'unit': {'type': 'string', 'enum': ['celsius', 'fahrenheit']}}, 'required': ['location']}, strict=None), type='function')], truncation_strategy=None, usage=None, temperature=N